In [2]:
from neo4j import GraphDatabase

# URI examples: "neo4j://localhost", "neo4j+s://xxx.databases.neo4j.io"


In [50]:
URI = "bolt://localhost:7687"
AUTH = ("neo4j", "azerty92")

with GraphDatabase.driver(URI, auth=AUTH) as driver: 
    driver.verify_connectivity() 

In [51]:
def create_person(tx, name):  
    result = tx.run(
        "MERGE (:Person {name: $name, school:'Telecom Paris'})",  
        name=name  
    )
    summary = result.consume()
    return summary

In [31]:
with driver.session(database="foo") as session:
    summary = session.execute_write(create_person, name="Guillaume")  

    print("Created {nodes_created} nodes in {time} ms.".format(
        nodes_created=summary.counters.nodes_created,
        time=summary.result_available_after
    ))

Created 0 nodes in 3 ms.


In [63]:
def count_row(tx, fn):  
    result = tx.run(
        f"""
        LOAD CSV FROM 'file:///{fn}' AS row FIELDTERMINATOR '\t'
        RETURN COUNT(row)
        """ 
    )
    count = result.fetch(1)
    summary = result.consume()
    return count, summary

def exec_write(query, csv_filename, db):
    # Execute a write query in the provided DB from a csv
    with driver.session(database=db) as session:
        summary = session.run(query.format(fn=csv_filename)).consume()

        print("Created {nodes_created} nodes and {relationship_created} relationships in {time} ms.".format(
            nodes_created=summary.counters.nodes_created,
            relationship_created=summary.counters.relationships_created,
            time=summary.result_available_after
        ))

# Create Events and actors

In [73]:
# Count lines in csv
with driver.session(database="foo") as session:
    count, summary = session.execute_write(count_row, fn='20230101000000.translation.export.csv')  

    print("Created {nodes_created} nodes in {time} ms.".format(
        nodes_created=summary.counters.nodes_created,
        time=summary.result_available_after
    ))
    print(count)

Created 0 nodes in 1 ms.
[<Record COUNT(row)=332>]


In [66]:
# Create event nodes
create_event = """
LOAD CSV FROM 'file:///{fn}' AS row FIELDTERMINATOR '\t'
CALL {{
    WITH row
    MERGE (event:Event {{GlobalEventID:row[0]}})
    ON CREATE
    SET
        event.Date = Datetime(row[1]),
        event.Type = row[26]
}} IN TRANSACTIONS OF 1000 ROWS;
"""

exec_write(create_event, "20230101000000.translation.export.csv", "foo")

Created 332 nodes and 0 relationships in 262 ms.


In [67]:
# Create actor1 node

# actor1Name idex: 6
# actor2Name idex: 16
create_actor1 = """ 
            LOAD CSV FROM 'file:///{fn}' AS row FIELDTERMINATOR '\t'
            WITH row WHERE row[6] IS NOT NULL
            CALL {{
                WITH row
                MERGE (:Actor {{Name:row[6]}})
            }} IN TRANSACTIONS OF 1000 ROWS;
        """
exec_write(create_actor1, "20230101000000.translation.export.csv", "foo")

Created 113 nodes and 0 relationships in 79 ms.


In [68]:
# Create actor2 node

# actor1Name idex: 6
# actor2Name idex: 16
create_actor2 = """ 
            LOAD CSV FROM 'file:///{fn}' AS row FIELDTERMINATOR '\t'
            WITH row WHERE row[16] IS NOT NULL
            CALL {{
                WITH row
                MERGE (:Actor {{Name:row[16]}})
            }} IN TRANSACTIONS OF 1000 ROWS;
        """
exec_write(create_actor2, "20230101000000.translation.export.csv", "foo")

Created 27 nodes and 0 relationships in 67 ms.


In [69]:
# Create relationship between actor1 and event

# actor1Name idex: 6
# actor2Name idex: 16
create_actor1_event = """
            LOAD CSV FROM 'file:///{fn}' AS row FIELDTERMINATOR '\t'
            WITH row WHERE row[6] IS NOT NULL
            MATCH (event:Event {{GlobalEventID : row[0]}})
            MATCH (actor:Actor {{Name : row[6]}})
            CALL {{
                WITH event, actor
                MERGE (actor)-[:GENERATES]->(event)
            }} IN TRANSACTIONS OF 1000 ROWS;
        """
exec_write(create_actor1_event, "20230101000000.translation.export.csv", "foo")

Created 0 nodes and 280 relationships in 365 ms.


In [70]:
# Create relationship between event and actor2

# actor1Name idex: 6
# actor2Name idex: 16
create_event_actor2 = """ 
            LOAD CSV FROM 'file:///{fn}' AS row FIELDTERMINATOR '\t'
            WITH row WHERE row[16] IS NOT NULL
            MATCH (event:Event {{GlobalEventID : row[0]}})
            MATCH (actor:Actor {{Name : row[16]}})
            CALL {{
                WITH event, actor
                MERGE (event)-[:IMPACTS]->(actor)
            }} IN TRANSACTIONS OF 1000 ROWS;
        """
exec_write(create_event_actor2, "20230101000000.translation.export.csv", "foo")

Created 0 nodes and 198 relationships in 151 ms.


# Create Mentions and Resources

In [81]:
create_resource = """ 
            LOAD CSV FROM 'file:///{fn}' AS row FIELDTERMINATOR '\t'
            WITH row WHERE row[5] IS NOT NULL
            CALL {{
                WITH row
                MERGE (:Resource {{OriginalID:row[5]}})
            }} IN TRANSACTIONS OF 1000 ROWS;
        """
exec_write(create_resource, "20230101000000.translation.mentions.csv", "foo")

Created 127 nodes and 0 relationships in 120 ms.


In [87]:
create_resource_event = """ 
            LOAD CSV FROM 'file:///{fn}' AS row FIELDTERMINATOR '\t'
            WITH row WHERE row[5] IS NOT NULL
            CALL {{
                WITH row
                MATCH (event:Event {{GlobalEventID : row[0]}})
                MATCH (resource:Resource {{OriginalID : row[5]}})
                MERGE (resource)-[:MENTIONS {{Confidence:coalesce(toInteger(row[11]), 0)}}]->(event)
            }} IN TRANSACTIONS OF 1000 ROWS;
        """
exec_write(create_resource_event, "20230101000000.translation.mentions.csv", "foo")

Created 0 nodes and 357 relationships in 527 ms.
